In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

FILE_PATH = 'glass.csv'                                # csv path
raw_data = pd.read_csv(FILE_PATH)						    

print("Raw data loaded successfully...\n")
#------------------------------------------------------------------------------
# 參數

Y_LABEL = 'Type'                                   		    	# 結果label
KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# 用來運算預測的key
N_INSTANCES = raw_data.shape[0]                     			# Number of instances
N_INPUT = raw_data.shape[1] - 1                     			# Input size
N_CLASSES = 8                                                   # 類型數
TEST_SIZE = 0.1                                    			    # 測試數
TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))     			# 訓練大小
LEARNING_RATE = 0.001                              			    # 學習rate
TRAINING_EPOCHS = 350                               			# epochs
BATCH_SIZE = 100                                    			# Batch size
DISPLAY_STEP = 1                                    			# Display progress each x epochs
HIDDEN_SIZE = 1000	                                   	    	# Number of hidden neurons
STDDEV = 0.1                                        			# Standard deviation (for weights random init)
RANDOM_STATE = 100 									            # Random state for train_test_split

#------------------------------------------------------------------------------
# Loading data

# Load data
data = raw_data[KEYS].get_values()                  			# X data
labels = raw_data[Y_LABEL].get_values()             			# y data
# One hot encoding for labels
labels_ = np.zeros((N_INSTANCES, N_CLASSES))
labels_[np.arange(N_INSTANCES), labels] = 1

# Train-test split
data_train, data_test, labels_train, labels_test = train_test_split(data,
                                                                    labels_,
                                                                    test_size = TEST_SIZE,
                                                                    random_state = RANDOM_STATE)

print("讀取完畢 檔案分割成功\n")
#------------------------------------------------------------------------------
#神經網路架構

# Tf placeholders
X = tf.placeholder(tf.float32, [None, N_INPUT])
y = tf.placeholder(tf.float32, [None, N_CLASSES])
dropout_keep_prob = tf.placeholder(tf.float32)

# 自定義層次函數
def layer(output_dim, input_dim, inputs, activation = None):
    W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    b = tf.Variable(tf.random_normal([1, output_dim]))
    XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs

# Build model
h1 = layer(output_dim = HIDDEN_SIZE, input_dim = N_INPUT, inputs = X, activation = tf.nn.tanh)
h2 = layer(output_dim = HIDDEN_SIZE, input_dim = HIDDEN_SIZE, inputs = h1, activation = tf.nn.tanh)
h3 = layer(output_dim = HIDDEN_SIZE, input_dim = HIDDEN_SIZE, inputs = h2, activation = tf.nn.tanh)
pred = layer(output_dim = N_CLASSES, input_dim = HIDDEN_SIZE, inputs = h3, activation = None)
#loss以及優化公式
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

#計算正確率
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#初始化
init = tf.initialize_all_variables()

print("開始訓練\n")
#------------------------------------------------------------------------------
# Training

sess = tf.Session()
sess.run(init)
loss_list = []
epoch_list = []
accuracy_list = []

#訓練迴圈
for epoch in range(TRAINING_EPOCHS):
    avg_cost = 0.
    total_batch = int(data_train.shape[0] / BATCH_SIZE)
    #batches
    for i in range(total_batch):
        randidx = np.random.randint(int(TRAIN_SIZE), size = BATCH_SIZE)
        batch_xs = data_train[randidx, :]
        batch_ys = labels_train[randidx, :]
        #優化
        sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.9})
        #平均loss
        avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
    #log
    if epoch % DISPLAY_STEP == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
        train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
        print ("Training accuracy: %.3f" % (train_acc))
        epoch_list.append(epoch)
        loss_list.append(avg_cost)
        accuracy_list.append(train_acc)



print ("訓練完成")
print("開始測試...\n")
#------------------------------------------------------------------------------
# Testing

test_acc = sess.run(accuracy, feed_dict={X: data_test, y: labels_test, dropout_keep_prob:1.})
print ("Test accuracy: %.3f" % (test_acc))

sess.close()
print("Session closed!")

#顯示曲線圖
#%matplotlib inline
fig = plt.gcf()
fig.set_size_inches(5, 3)
plt.plot(epoch_list, loss_list, label = 'loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss'], loc = 'upper right')
plt.show()

fig = plt.gcf()
fig.set_size_inches(5, 3)
plt.plot(epoch_list, accuracy_list, label = 'accuracy')

plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy'], loc = 'upper left')
plt.show()

Raw data loaded successfully...

讀取完畢 檔案分割成功

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
開始訓練

Epoch: 000/350 cost: 12.582519531
Training accuracy: 0.300
Epoch: 001/350 cost: 10.318268776
Training accuracy: 0.530
Epoch: 002/350 cost: 10.282422066
Training accuracy: 0.540
Epoch: 003/350 cost: 7.890991688
Training accuracy: 0.600
Epoch: 004/350 cost: 4.626994133
Training accuracy: 0.720
Epoch: 005/350 cost: 6.384871960
Training accuracy: 0.600
Epoch: 006/350 cost: 7.354812622
Training accuracy: 0.570
Epoch: 007/350 cost: 4.385900974
Training accuracy: 0.770
Epoch: 008/350 cost: 7.666756153
Training accuracy: 0.600
Epoch: 009/350 cost: 3.206178904
Training accuracy: 0.710
Epoch: 010/350 cost: 4.964400768
Training accuracy: 0.650
Epoch: 011/350 cost: 3.254843950
Training ac

Training accuracy: 0.960
Epoch: 130/350 cost: 0.240266249
Training accuracy: 0.910
Epoch: 131/350 cost: 0.178478867
Training accuracy: 0.910
Epoch: 132/350 cost: 0.218792155
Training accuracy: 0.900
Epoch: 133/350 cost: 0.207554117
Training accuracy: 0.930
Epoch: 134/350 cost: 0.158463970
Training accuracy: 0.950
Epoch: 135/350 cost: 0.135016799
Training accuracy: 0.920
Epoch: 136/350 cost: 0.302147031
Training accuracy: 0.910
Epoch: 137/350 cost: 0.189288870
Training accuracy: 0.970
Epoch: 138/350 cost: 0.184369549
Training accuracy: 0.920
Epoch: 139/350 cost: 0.149155259
Training accuracy: 0.950
Epoch: 140/350 cost: 0.178555757
Training accuracy: 0.940
Epoch: 141/350 cost: 0.183086678
Training accuracy: 0.910
Epoch: 142/350 cost: 0.386295080
Training accuracy: 0.900
Epoch: 143/350 cost: 0.165251791
Training accuracy: 0.950
Epoch: 144/350 cost: 0.125727355
Training accuracy: 0.950
Epoch: 145/350 cost: 0.156072468
Training accuracy: 0.950
Epoch: 146/350 cost: 0.106329463
Training accur

Epoch: 274/350 cost: 0.283169627
Training accuracy: 0.900
Epoch: 275/350 cost: 0.240085989
Training accuracy: 0.940
Epoch: 276/350 cost: 0.660145521
Training accuracy: 0.860
Epoch: 277/350 cost: 0.374904662
Training accuracy: 0.900
Epoch: 278/350 cost: 0.636112690
Training accuracy: 0.790
Epoch: 279/350 cost: 0.612725317
Training accuracy: 0.730
Epoch: 280/350 cost: 0.494627506
Training accuracy: 0.810
Epoch: 281/350 cost: 0.485589832
Training accuracy: 0.850
Epoch: 282/350 cost: 0.705573738
Training accuracy: 0.770
Epoch: 283/350 cost: 0.438620448
Training accuracy: 0.860
Epoch: 284/350 cost: 0.429703295
Training accuracy: 0.860
Epoch: 285/350 cost: 0.507867992
Training accuracy: 0.830
Epoch: 286/350 cost: 0.367292494
Training accuracy: 0.890
Epoch: 287/350 cost: 0.399346322
Training accuracy: 0.900
Epoch: 288/350 cost: 0.495057464
Training accuracy: 0.850
Epoch: 289/350 cost: 0.101174161
Training accuracy: 0.980
Epoch: 290/350 cost: 0.353420287
Training accuracy: 0.880
Epoch: 291/350